In [ ]:
!pip install -U -qq transformers accelerate sentence-transformers

In [ ]:
from huggingface_hub import notebook_login, Repository

# Login to Hugging Face
notebook_login()

### GPU
1. P100: Powerful and suited for training complex models due to its high memory (16GB) but consumes more power (250W). 
2. T4 x2: More energy-efficient (70W) with decent memory (16GB) making it ideal for inference (using trained models) and less complex training tasks. Having two T4s doubles the processing power.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import time

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto") 
                                             
tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          use_fast=True) 

In [ ]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

#### Checklist: Before you start writing your cover letter, prepare
1. Information about the company, role, and industry specifics;
2. A tailored resume that aligns with the job you’re applying for;
3. Company-specific instructions or preferences for the style of writing and what would be counted as impressive achievements (e.g. assess what is the company culture and the type of candidate they’re looking for);
4. How do you hope to communicate your career goals within the cover letter;
5. Other company information that is relevant to the application (values and mission, recent accomplishments, etc.);
6. Patience and time.

#### Main purpose
is to catch recruiters' interests by finding the balance between:
* dedicated cover letter space and filler words/ language;
* key skills or achievements without repeating candidate resumes;
* showing authentic interest in the role while highlighting job keywords;
* being specific and personable, yet ticking all the requirement boxes by allowing recruiters to imagine what the candidates' potential would be if they joined the company.

In [ ]:
resume = """


"""


job_description = """


"""

### Problem Solving Approach

In [ ]:
resp_messages = [
{"role": "system", 
"content": """

I will provide you with the job description and my resume. 
Use this information to create a professional and effective cover letter. 
Focus the cover letter to be written using the best practices and industry standards. 
Tailor the cover letter's content to the specific job and company that I am applying to, 
highlighting my relevant skills and experience. 
Also, make sure to explain why I am a strong candidate for the position. 


I'd like you to use the following problem-solving structure to write the cover letter. 
In the first paragraph, mention one specific challenge that this company may face, which the role can solve. 
The next paragraph should focus on how up to three skills 
and one of my past professional roles make me the perfect candidate to resolve this problem. 

Finalize with more information about why I'm the perfect candidate for the role. 

Use professional language and tone, but at the same time keep this cover letter compact. 
Follow the best cover letter writing practices.

Please ensure that the cover letter is clear and concise. 
The cover letter should have appropriate formatting and layout to make it visually appealing and easy to read. 
Do make sure that it effectively communicates my qualifications and interest in the job. 
Finally, don't include any personal opinions or statements.

"""},
{"role": "user", 
"content": f"""

## Position 
{position} 

Here’s my resume: 
{resume}

Here’s the job I’m applying for: 
{job_description}

Could you please write me a cover letter?


"""},
]
start = time.time()

resp_input_ids = tokenizer.apply_chat_template(
    resp_messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda:0")

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

resp_outputs = model.generate(
    resp_input_ids,
    max_new_tokens=4096,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
resp_response = resp_outputs[0][resp_input_ids.shape[-1]:]
print(tokenizer.decode(resp_response, skip_special_tokens=True))
end = time.time()
print(f"Time (minutes): {(end - start)}")